In [ ]:
from notebook_prelude import *

In [ ]:
phi_files = glob('data/CACHE/*.fast_wl.*.npy')
for phi_file in phi_files:
    parts = phi_file.split('.')
    labels = parts[-2]
    metric = parts[-3]
    phi_lists, Y = dataset_helper.get_dataset_cached(phi_file, check_validity=False)
    num_phis = len(phi_lists)
    print(num_phis, labels, metric, phi_file)

In [ ]:
from transformers.phi_picker_transformer import PhiPickerTransformer
from transformers.pipelines.classifiers import get_classifier_params
import sklearn.pipeline
import sklearn.model_selection

phi_picker = PhiPickerTransformer()

estimator = sklearn.pipeline.Pipeline([
    #('phi_picker', phi_picker),
    ('normalizer', sklearn.preprocessing.MaxAbsScaler()),
    ('classifier', None)
])
param_grid = dict()
param_grid = dict(param_grid, **get_classifier_params())
cv = sklearn.model_selection.StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

gscv = sklearn.model_selection.GridSearchCV(estimator, param_grid=param_grid, scoring='f1_macro', cv=cv, verbose=11)

In [ ]:
phi_files = glob('data/CACHE/*.fast_wl.*.npy')
for phi_file in phi_files:
    dataset = filename_utils.get_dataset_from_filename(phi_file)
    if dataset not in ['ng20']: continue
    if 'same_label' in phi_file: continue
    if 'concept' not in phi_file: continue
    if 'none' not in phi_file: continue
    print(phi_file)
    phi_lists, Y = dataset_helper.get_dataset_cached(phi_file, check_validity=False)
    gscv_result = gscv.fit(phi_picker.transform(phi_lists), Y)
    break

In [ ]:
pd.DataFrame(gscv_result.cv_results_)

In [ ]:
phi_file